In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pyjags


/Users/yinx/opt/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
# Load the NFL concussions data
Y = np.array([171, 152, 123, 199])
n = 4
N = 256

In [3]:
# Define the model as a string
model_string = '''
model{
  
  # Likelihood
  for(i in 1:n){
    Y[i] ~ dpois(N * lambda[i])
  }
  
  # Priors
  for(i in 1:n){
    lambda[i] ~ dgamma(1, gamma)
  }
  gamma ~ dgamma(a, b)
}
'''

In [4]:
# Load the data and compile the MCMC code

inits = {'lambda': np.random.gamma(1, 1, n), 'gamma': 1} # initial values for lambda and gamma
data = {'Y': Y, 'N': N, 'n': n, 'a': 0.1, 'b': 0.1} # data should be stored in a dictionary
# model = pyjags.Model(model_string, data=data, init=inits, chains=2)
model = pyjags.Model(model_string, data= dict(data), init=dict(inits), chains=2)

: 

: 

In [ ]:
# Burn-in for 10000 samples
model.update(10000)

# Generate 20000 post-burn-in samples
samples = model.sample(20000)

In [ ]:
# Produce a summary for the MCMC samples

summary = pd.DataFrame(samples['lambda'])
summary.columns = ['lambda1', 'lambda2', 'lambda3', 'lambda4']
print(summary.describe())


In [ ]:
# Diagnostic checks
plt.figure(figsize=(10, 6))
plt.subplot(2, 2, 1)
plt.plot(samples['lambda'][:, 0])
plt.xlabel('Iteration')
plt.ylabel('lambda1')
plt.subplot(2, 2, 2)
plt.plot(samples['lambda'][:, 1])
plt.xlabel('Iteration')
plt.ylabel('lambda2')
plt.subplot(2, 2, 3)
plt.plot(samples['lambda'][:, 2])
plt.xlabel('Iteration')
plt.ylabel('lambda3')
plt.subplot(2, 2, 4)
plt.plot(samples['lambda'][:, 3])
plt.xlabel('Iteration')
plt.ylabel('lambda4')
plt.tight_layout()
plt.show()

plt.figure(figsize=(10, 6))
plt.subplot(2, 2, 1)
plt.hist(samples['lambda'][:, 0], bins=20)
plt.xlabel('lambda1')
plt.subplot(2, 2, 2)
plt.hist(samples['lambda'][:, 1], bins=20)
plt.xlabel('lambda2')
plt.subplot(2, 2, 3)
plt.hist(samples['lambda'][:, 2], bins=20)
plt.xlabel('lambda3')
plt.subplot(2, 2, 4)
plt.hist(samples['lambda'][:, 3], bins=20)
plt.xlabel('lambda4')
plt.tight_layout()
plt.show()

plt.figure(figsize=(10, 6))
plt.subplot(2, 2, 1)
plt.plot(samples['lambda'][:, 0])
plt.xlabel('Iteration')
plt.ylabel('lambda1')
plt.subplot(2, 2, 2)
plt.plot(samples['lambda'][:, 1])
plt.xlabel('Iteration')
plt.ylabel('lambda2')
plt.subplot(2, 2, 3)
plt.plot(samples['lambda'][:, 2])
plt.xlabel('Iteration')
plt.ylabel('lambda3')
plt.subplot(2, 2, 4)
plt.plot(samples['lambda'][:, 3])
plt.xlabel('Iteration')
plt.ylabel('lambda4')
plt.tight_layout()
plt.show()

In [ ]:
# Compute 90% credible intervals
samps = np.vstack((samples['lambda'][:, 0], samples['lambda'][:, 1], samples['lambda'][:, 2], samples['lambda'][:, 3])).T
cred_int = np.apply_along_axis(lambda x: np.quantile(x, [0.05, 0.95]), axis=0, arr=samps)
print(cred_int)